In [1]:
from mesa import Agent, Model
from mesa.discrete_space import CellAgent, FixedAgent

# Analysis
from mesa.datacollection import DataCollector
from mesa.space import MultiGrid

C:\Users\trine\miniconda3\envs\cb1020\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
class Bloodcell(CellAgent):
    """The Bloodcell class."""

    def __init__(
        self, model, healthpoint=10, infection_risk=0.5
    ):
        """Initialize an Bloodcell.

        Args:
            model: Model instance
            healthpoint: Starting healthpoint
            infection_risk: Risk of getting infected when encountering a Merozoite
        """
        super().__init__(model)
        self.hp = healthpoint
        self.i_risk = infection_risk
        self.alive = True

    def step(self):
        """Execute one step of the blodcell lifespan """

        # If infected,  the cell dies and after 10 steps it bursts and releases merozoites, is then removed
        if self.hp <= 9:
            self.alive = False # Nu död
            self.hp -= 1
            
        if self.hp <= 0:
            self.remove()

class Sicklecell(CellAgent):
    """The Sicklecell class."""

    def __init__(
        self, model, healthpoint=10, infection_risk=0.5
    ):
        """Initialize an Sicklecell.

        Args:
            model: Model instance
            healthpoint: Starting healthpoint
            infection_risk: Risk of getting infected when encountering a Merozoite
        """
        super().__init__(model)
        self.hp = healthpoint
        self.i_risk = infection_risk
        self.alive = True
       
    def step(self):
        """Execute one step of the blodcell lifespan """

        # Sickels right away when infected, is then removed
        if self.hp <= 9:
            self.alive = False # Nu död
            self.remove()



class Merozoite(CellAgent):
    """A Merozoite that swims around, reproduces (asexually) and infects blood cells."""
    def __init__(
        self, model, reproducepoint=10
    ):
        """Initialize an Merozoite.

        Args:
            model: Model instance
            reproducepoint: Starting reproducepoint
        """
        
        super().__init__(model)
        self.rp = reproducepoint
        self.alive = True

    def spawnoffspring(self):
        new_agent = Merozoite(
        self.model,
        pos=self.pos,
        reproducepoint=self.rp
        )
        self.model.grid.place_agent(new_agent, self.pos)
        self.model.agents_by_type[Merozoite].add(new_agent)
        self.model.agents.add(new_agent)
        return new_agent

        
    def infect(self):
        """If possible, infect bloodcell at current location."""
        Bloodcell = [obj for obj in self.cell.agents if isinstance(obj, Bloodcell)]
        if Bloodcell:  # If there are any sheep present
            Bloodcell_to_eat = self.random.choice(Bloodcell)
            Bloodcell_to_eat.hp -= 1 #kanske inte parenteser
            self.rp -= 1
   
    def step(self):
        """Execute one step of the Merozoite lifespan """
        # Move to random neighboring cell
        self.move()
    
        # When infecting, rp goes down, after 10 steps offsprings is released 
        if self.rp <= 9:
            self.alive = False # Nu död
            self.rp -= 1
            
        if self.rp <= 0:
            self.spawnoffspring()
            
    def move(self):
        """Move to a neighboring cell"""
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,  # current position
            moore=True,  # including step on the diagonal
            include_center=False,  # to remain at current position is not allowed
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)


In [48]:
import math

from mesa import Model
from mesa.datacollection import DataCollector
from mesa.discrete_space import OrthogonalVonNeumannGrid
from mesa.space import MultiGrid
from mesa.examples.advanced.wolf_sheep.agents import GrassPatch, Sheep, Wolf
from mesa.experimental.devs import ABMSimulator


class Malaria(Model):
    """Malaria Model.

    A model for simulating wolf and sheep (predator-prey) ecosystem modelling.
    """

    description = (
        "A model for simulating wolf and sheep (predator-prey) ecosystem modelling."
    )

    def __init__(
        self,
        width=10,
        height=10,
        initial_Bloodcell=70,
        initial_Sicklecell=30,
        initial_Merozoite=5,
        self.datacollector = DataCollector(# den data vi vill samla för 
            model_reporters = { 
                "Number of Bloodcells" : # hur många?,
                "Number of Sicklecells" : # hur många?,
                "Number of Merozoites" : # hur många?,
            },
            agent_reporters = {
                "Agent position" : "pos"
                "Bloodcell healthpoint" : "healthpoint",
            }
        
        seed=None,
        #simulator: ABMSimulator = None,
    ):
        """Create a new Wolf-Sheep model with the given parameters.

        Args:
            height: Height of the grid
            width: Width of the grid
            initial_sheep: Number of sheep to start with
            initial_wolves: Number of wolves to start with
            sheep_reproduce: Probability of each sheep reproducing each step
            wolf_reproduce: Probability of each wolf reproducing each step
            wolf_gain_from_food: Energy a wolf gains from eating a sheep
            grass: Whether to have the sheep eat grass for energy
            grass_regrowth_time: How long it takes for a grass patch to regrow
                                once it is eaten
            sheep_gain_from_food: Energy sheep gain from grass, if enabled
            seed: Random seed
            simulator: ABMSimulator instance for event scheduling
        """
        super().__init__(seed=seed)
        #self.simulator = simulator
        #self.simulator.setup(self)

        # Initialize model parameters
        self.height = height
        self.width = width

        # Create grid using experimental cell space
        self.grid = MultiGrid(width, height, True)

        # Set up data collection
        model_reporters = {
            "Bloodcell": lambda m: len(m.agents_by_type[Bloodcell]),
            "Sicklecell": lambda m: len(m.agents_by_type[Sicklecell]),
            "Merozoite": lambda m: len(m.agents_by_type[Merozoite]),
        }

        self.datacollector = DataCollector(model_reporters)

        for _ in range(initial_Bloodcell):
            agent = Bloodcell(self)
            # self.schedule.add(agent)

            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))

        for _ in range(initial_Sicklecell):
            agent = Sicklecell(self)
            # self.schedule.add(agent)

            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))

        for _ in range(initial_Merozoite):
            agent = Merozoite(self)
            # self.schedule.add(agent)

            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))


        # Collect initial data
        self.running = True
        self.datacollector.collect(self)

    def step(self):
        """Execute one step of the model."""
        self.datacollector.collect(self)
        # First activate all sheep, then all wolves, both in random order
        self.agents_by_type[Bloodcell].shuffle_do("step")
        self.agents_by_type[Sicklecell].shuffle_do("step")
        self.agents_by_type[Merozoite].shuffle_do("step")

        # Collect data
        self.datacollector.collect(self)

In [49]:
model = Malaria()

for i in range(100):
    model.step()

In [50]:
model_df = model.datacollector.get_model_vars_dataframe()
agent_df = model.datacollector.get_agent_vars_dataframe()

UserWarning: No agent reporters have been defined in the DataCollector, returning empty DataFrame.